In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_131154/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 07:06:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 07:06:54 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})


df_person = spark.createDataFrame(employee)
df_person.createOrReplaceTempView("Employee")

In [4]:
employee

,id,name,department,managerId
0,101,John,A,<NA>
1,102,Dan,A,101
2,103,James,A,101
3,104,Amy,A,101
4,105,Anne,A,101
5,106,Ron,B,101


In [5]:
query = """
    select
        *
    from
        Employee e
    where 
        (select count(distinct id) from Employee e1 where managerId = e.id) >= 5
"""

In [6]:
spark.sql(query).show()

+---+----+----------+---------+
| id|name|department|managerId|
+---+----+----------+---------+
|101|John|         A|      NaN|
+---+----+----------+---------+

